In [1]:
import rasterio
from rasterio.mask import mask
from scipy.ndimage import generic_filter
import geopandas as gpd
import numpy as np
from pyproj import Proj, transform
import pandas as pd

In [2]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

portugal = world[world['name'] == 'Portugal']['geometry'].values[0]

print(portugal)

POLYGON ((-9.034817674180246 41.880570583659676, -8.67194576662672 42.13468943945496, -8.263856980817792 42.28046865495034, -8.013174607769912 41.790886135417125, -7.422512986673795 41.79207469335984, -7.251308966490824 41.91834605566505, -6.6686055159676565 41.883386949219584, -6.389087693700915 41.381815497394655, -6.851126674822552 41.11108266861753, -6.864019944679385 40.33087189387483, -7.026413133156595 40.184524237624245, -7.066591559263529 39.711891587882775, -7.498632371439726 39.62957103124181, -7.098036668313128 39.03007274022379, -7.374092169616318 38.37305858006492, -7.029281175148796 38.07576406508977, -7.166507941099865 37.803894354802225, -7.537105475281024 37.42890432387624, -7.453725551778092 37.09778758396607, -7.855613165711986 36.83826854099627, -8.382816127953689 36.97888011326246, -8.898856980820327 36.86880931248078, -8.746101446965554 37.65134552667661, -8.83999752443988 38.266243394517616, -9.287463751655224 38.3584858261586, -9.526570603869715 38.737429104154

/tmp/ipykernel_17933/3069684263.py:1: FutureWarning: The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.
  world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))


In [ ]:
NaturalFires = pd.read_csv('NaturalFires.csv')

In [3]:
with rasterio.open('md75_grd/md75_grd/w001001.adf') as src:
    # Clip the raster to the extent of Portugal
    out_image, out_transform = mask(src, [portugal.__geo_interface__], crop=True)
    out_meta = src.meta.copy()
    out_meta.update({"driver": "GTiff",
                     "height": out_image.shape[1],
                     "width": out_image.shape[2],
                     "transform": out_transform})

# Save the clipped raster
with rasterio.open('clipped.tif', 'w', **out_meta) as dest:
    dest.write(out_image)

# Now you can calculate the TRI on the clipped raster
with rasterio.open('clipped.tif') as src:
    # Read the raster data
    raster_data = src.read(1)

    def calculate_tri(dem):
        def tri_filter(x):
            central_value = x[len(x) // 2]
            return np.mean(np.abs(x - central_value))

        return generic_filter(dem, tri_filter, size=3)

    # Calculate TRI
    tri = calculate_tri(raster_data)


longitude = 37.283231
latitude = -8.5772292

# Define the source CRS (WGS84)
src_crs = Proj(init='EPSG:4326')

# Get the CRS of the raster data
dst_crs = Proj(src.crs)

# Convert the coordinates to the raster data's CRS
longitude_proj, latitude_proj = transform(src_crs, dst_crs, longitude, latitude)

# Convert the projected coordinates to raster indices
row, col = src.index(longitude_proj, latitude_proj)

# Check if the indices are within the dimensions of the array
if 0 <= row < tri.shape[0] and 0 <= col < tri.shape[1]:
    tri_value = tri[row, col]
    print(tri_value)
else:
    print(f"The coordinates ({longitude}, {latitude}) are outside the extent of the raster.")

17


/home/ori/.local/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/tmp/ipykernel_17933/44699290.py:42: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  longitude_proj, latitude_proj = transform(src_crs, dst_crs, longitude, latitude)


In [4]:
from osgeo import gdal
from pyproj import Proj, transform

longitude = 37.283231
latitude = -8.5772292

# Open the DEM file
dem = gdal.Open('clipped.tif')  # replace 'clipped.tif' with your DEM file

# Calculate the slope
gdal.DEMProcessing('slope.tif', dem, 'slope')

# Close the DEM file
dem = None

# Open the slope file
slope_ds = gdal.Open('slope.tif')

# Get the CRS of the raster data
dst_crs = Proj(slope_ds.GetProjection())

# Define the source CRS (WGS84)
src_crs = Proj(init='EPSG:4326')

# Convert the coordinates to the raster data's CRS
longitude_proj, latitude_proj = transform(src_crs, dst_crs, longitude, latitude)

# Convert the projected coordinates to raster indices
gt = slope_ds.GetGeoTransform()
col = int((longitude_proj - gt[0]) / gt[1])
row = int((latitude_proj - gt[3]) / gt[5])

# Read the slope data
slope_data = slope_ds.ReadAsArray()

# Check if the indices are within the dimensions of the array
if 0 <= row < slope_data.shape[0] and 0 <= col < slope_data.shape[1]:
    slope_value = slope_data[row, col]
    print(slope_value)
else:
    print(f"The coordinates ({longitude}, {latitude}) are outside the extent of the raster.")

# Close the slope file
slope_ds = None

89.994255


/home/ori/.local/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/tmp/ipykernel_17933/3793152031.py:26: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  longitude_proj, latitude_proj = transform(src_crs, dst_crs, longitude, latitude)


In [9]:
import rasterio

# Open the raster file
with rasterio.open('clipped.tif') as src:
    # Get the cell size
    cellsize_x, cellsize_y = src.res

print('Cell size in x direction:', cellsize_x)
print('Cell size in y direction:', cellsize_y)

Cell size in x direction: 0.00833333333333337
Cell size in y direction: 0.00833333333333337


In [10]:
def calculate_slope(dem, cellsize):
    def slope_filter(window):
        # Reshape the window to a 3x3 array
        matrix = window.reshape((3, 3))

        # Calculate the derivatives in the x and y directions
        dz_dx = (matrix[1, 2] - matrix[1, 0]) / (2 * cellsize)
        dz_dy = (matrix[2, 1] - matrix[0, 1]) / (2 * cellsize)

        # Calculate the slope
        slope = np.sqrt(dz_dx**2 + dz_dy**2)

        return slope

    return generic_filter(dem, slope_filter, size=3)

# Calculate slope
cellsize = 0.00833333333333337  # replace with your actual cell size
slope = calculate_slope(raster_data, cellsize)

In [16]:
def calculate_slope(dem):
    # Calculate the derivatives in the x and y directions
    dz_dx = (dem[1, 2] - dem[1, 0]) / 2
    dz_dy = (dem[2, 1] - dem[0, 1]) / 2

    # Calculate the slope
    slope = np.sqrt(dz_dx**2 + dz_dy**2)

    return slope

def calculate_slope_at_coordinate(dem, lon, lat, transform):
    # Convert the geographic coordinates to the raster's coordinate system
    col, row = ~transform * (lon, lat)

    # Make sure the row and column are integers
    col = int(col)
    row = int(row)

    # Check if the coordinate is within the bounds of the raster
    if row <= 0 or row >= dem.shape[0] - 1 or col <= 0 or col >= dem.shape[1] - 1:
        raise ValueError('The coordinate is on the edge of the raster or outside the raster')

    # Extract a 3x3 window around the cell
    window = dem[row-1:row+2, col-1:col+2]

    # Calculate the slope for the window
    slope = calculate_slope(window)

    return slope


# Get the raster's transform
with rasterio.open('clipped.tif') as src:
    transform = src.transform

# Calculate the slope at the specific geographic coordinate
lon = 37.283231  # replace with your actual longitude
lat = -8.5772292  # replace with your actual latitude
slope = calculate_slope_at_coordinate(raster_data, lon, lat, transform)
print(slope)

ValueError: The coordinate is on the edge of the raster or outside the raster

In [17]:
with rasterio.open('clipped.tif') as src:
    transform = src.transform
    bounds = src.bounds

# Check if the coordinate is within the bounds of the raster
lon = 37.283231  # replace with your actual longitude
lat = -8.5772292  # replace with your actual latitude
if not (bounds.left < lon < bounds.right and bounds.bottom < lat < bounds.top):
    raise ValueError('The coordinate is outside the raster')

# Calculate the slope at the specific geographic coordinate
slope = calculate_slope_at_coordinate(raster_data, lon, lat, transform)
print(slope)

ValueError: The coordinate is outside the raster